In [3]:
import pandas as pd
import numpy as np

In [4]:
car=pd.read_csv('cost1.csv')

In [5]:
car.head()

,state,district,commodity,variety,modal_price
0,"""--Select State--""","""--Select District--""","""--Select Commodity--""","""--Select Variety--""",0.0
1,Assam,Barpeta,Black Gram (Urd Beans),Black Gram (Whole),2900.0
2,Assam,Barpeta,Corriander seed,Coriander Seed,3825.0
3,Assam,Barpeta,Jute,TD-5,3825.0
4,Assam,Barpeta,Lentil(Masur),Masoor Gola,3500.0


In [6]:
car.shape

(7419, 5)

In [7]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7419 entries, 0 to 7418
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   state        7419 non-null   object 
 1   district     7419 non-null   object 
 2   commodity    7419 non-null   object 
 3   variety      7419 non-null   object 
 4   modal_price  7419 non-null   float64
dtypes: float64(1), object(4)
memory usage: 289.9+ KB


In [11]:
car.describe()

,modal_price
count,7419.000000
mean,2803.762816
std,3916.914942
min,0.000000
25%,1000.000000
50%,1827.000000
75%,3500.000000
max,100000.000000


In [12]:
X= car.drop(columns='modal_price')
y= car['modal_price']

In [30]:
X

,state,district,commodity,variety
0,"""--Select State--""","""--Select District--""","""--Select Commodity--""","""--Select Variety--"""
1,Assam,Barpeta,Black Gram (Urd Beans),Black Gram (Whole)
2,Assam,Barpeta,Corriander seed,Coriander Seed
3,Assam,Barpeta,Jute,TD-5
4,Assam,Barpeta,Lentil(Masur),Masoor Gola
...,...,...,...,...
7414,West Bengal,Uttar Dinajpur,Jute,TD-5
7415,West Bengal,Uttar Dinajpur,Potato,Jyoti
7416,West Bengal,Uttar Dinajpur,Rice,Other
7417,West Bengal,Uttar Dinajpur,Jute,TD-5


In [31]:
y

0          0.0
1       2900.0
2       3825.0
3       3825.0
4       3500.0
         ...  
7414    3800.0
7415     850.0
7416    3300.0
7417    3700.0
7418    3200.0
Name: modal_price, Length: 7419, dtype: float64

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2)

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [15]:
ohe=OneHotEncoder()
ohe.fit(X[['state','district','commodity','variety']])

OneHotEncoder()

In [16]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['state','district','commodity','variety']),
                                        remainder='passthrough')

In [17]:
lr=LinearRegression()

In [18]:
pipe=make_pipeline(column_trans,lr)

In [19]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['"--Select State--"', 'Assam', 'Chattisgarh', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala',
       'Madhya Pradesh', 'Maharashtra', 'Meghalaya', 'NCT of Delhi',
       'Orissa', 'Pondicherry', 'Punjab', 'Rajast...
       'Sweet Pumpkin', 'TD-5', 'Talu', 'Tamarind Fruit', 'Tamarind Seed',
       'Tapioca', 'Tattibettee', 'Tender Coconut', 'Tinda', 'Tomato',
       'Totapuri', 'U.P. (Split)', 'UP', 'Unde', 'Ungrabled',
       'Water Melon', 'Wheat Atta', 'White', 'White Peas', 'White Ponni',
       'Whole', 'Yam', 'Yellow', 'Yellow (Black)', 'api', 'other'],
      dtype=object)]),
                                                  ['state', 'district',
                                                   'com

In [20]:
y_pred= pipe.predict(X_test)

In [21]:
r2_score(y_test,y_pred)

0.5856696298866226

In [22]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)                                               
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [23]:
np.argmax(scores)

976

In [24]:
scores[np.argmax(scores)]

0.9098653260925791

In [25]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)                                               
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.9098653260925791

In [26]:
import pickle

In [27]:
pickle.dump(car,open('df.pkl','wb'))
pickle.dump(pipe,open('LinearRegressionModel1.pkl','wb'))

In [28]:
pipe.predict(pd.DataFrame([['Karnataka','Davangere','Black Gram (Urd Beans)','Black Gram (Whole)']], columns=['state','district','commodity','variety']))

array([3345.24829981])